# $Telco$ $Churn$

# Goals:

* Discover the drivers of customer churn within the Telco company.

* Use drivers to built models that can predict if a Telco customer will churn.

# Imports

In [1]:
import pandas as pd
import numpy as np
import wrangle as w

# Acquire

* Data acquire from Codeup Database

* It contained  7043 rows and 24 columns before cleaning

* Each row represents a Telco customers

* Each column represents a feature related to the customer.

In [2]:
# acquire telco data 
df = w.get_telco_data()

# Prepare

prepare actions:

* Removed columns that contain unuseful information.
* Removed columns that contain redundant information.
* Removed 11 observations with whitespace in total charges column.
* Total charges column will be converted to a numeric column.
* Encoded categorical variables
* Split data into train, validate and test (approx: train 56%, validate 24%, test 20%), stratifying on 'churn'
* 


###                                                             Data Dictionary


| Feature | Definition |
| --- | --- |
| Gender | Whether a customer is male or female |
| Senior Citizen | Whether a customer is Senior Citizen (1) or not (0) |
| Partner | Whether the customer has a partner or not |
| Dependents | Whether the customer has dependents or not |
| Tenure | Number of months the customer has stayed with the company |
| PhoneService | Whether the customer has a phone service or not |
| MultipleLines | Whether the customer has multiple lines or not (Yes, No, No phone service)|
| InternetService | Customer’s internet service provider (DSL, Fiber optic, No internet srvice)|
| OnlineSecurity | Whether the customer has online security or not (Yes, No, No internet service) |
| Additional Features | Encoded and values for categorical data and scaled versions continuous data|

In [3]:
# cleaning data
df = w.prep_telco(df)

# Looking at the first rows of data

In [4]:
train.head(10)

NameError: name 'train' is not defined

# Data Summary

In [ ]:
train.describe()

# Explore